In [1]:
import requests
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')
import time
from fake_useragent import UserAgent
ua = UserAgent()

In [16]:
# cookie might need update
# Filled all Header from Jindon Website request headers
header = {
'Accept': '*/*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': '', 
'Connection': 'keep-alive',
'Cookie': '',
'Host': 'club.jd.com',
'Referer': 'https://item.jd.com/',
'sec-ch-ua': '',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': "",
'Sec-Fetch-Dest': 'script',
'Sec-Fetch-Mode': 'no-cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': ua.random}

In [17]:
def todict(text):
    sidx = text.find('{')
    eidx = text.rfind('}')
    return json.loads(text[sidx:eidx+1])

pid_files = open('Jindon_pid.txt','r')
pid_dic = eval(str(pid_files.read()))
pid_dic.keys()

In [27]:
df = pd.DataFrame()
for model in pid_dic.keys():
    pid = pid_dic[model]
    for page in range(100):  # Numbers of review pages
        try:
            tmp = pd.DataFrame()
            page = page
            url = f'https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId={pid}&score=0&sortType=6&page={page}&pageSize=10&isShadowSku=0&fold=1'
            response = requests.get(url, headers=header)

            print(f'{model} Page: {page} response: {response}', end='\r')

            text = response.content.decode('gbk')

            # Information you want please find in text
            require = ['id', 'content', 'creationTime',
                       'score', 'usefulVoteCount'] 

            content = todict(text)
            for i in range(len(content['comments'])):
                tmp = tmp.append({w: content['comments'][i][w]
                                 for w in require}, ignore_index=True)
                tmp['model'] = model
            df = df.append(tmp)
        except:
            if page == 100:
                print(f'{model} Page: {page} response: End', end='\r')
            else:
                print('Error')
            continue

        time.sleep(3)

,id,content,creationTime,score,usefulVoteCount,model
0,17274166986,投影亮度：好\n投影色彩：好\n音质音效：好\n操作难易：易,2022-03-20 17:06:36,5,0,長虹_T7U
1,17169237386,很清晰，白天看也看的清楚，高端大气上档次，效果好！军工品质就是不一样的，还有5年质保！,2022-02-23 19:35:16,5,0,長虹_T7U
2,17148387541,很清晰，客服也很有耐心，都说很划算，性价比特别高！,2022-02-19 12:32:32,5,0,長虹_T7U
3,17144798008,客服非常有耐心解答问题，安装师傅也特别优秀！专业，细心！上墙效果非常好，朋友都说我买的好。哈...,2022-02-18 16:53:52,5,0,長虹_T7U
4,17114184184,机子还没调好 播放起来感觉很清晰 投的还是白墙 噪音可以忽略 音效和电视剧差不多 总体感...,2022-02-11 21:55:37,5,0,長虹_T7U
...,...,...,...,...,...,...
9,15652338991,质量很好，价格很划算，老板发货很快，非常满意,2021-05-09 17:51:21,5,0,Epson_EH-LS300B
0,15640774011,服务很周到,2021-05-06 17:21:35,5,0,Epson_EH-LS300B
1,15639653029,投影仪收到了，质量很好，安装好了，投影效果很棒，画质清晰，流畅，色彩很真实，音质也特别好，特...,2021-05-06 12:29:48,5,0,Epson_EH-LS300B
2,15637983166,质量不错，发货很快包装严密没有磕碰，效果很好，直接发图。,2021-05-05 22:21:51,5,0,Epson_EH-LS300B
